# Kotlin Coroutines: Exceptions and Error Handling

This notebook explains how exceptions propagate in Kotlin coroutines and how to handle them effectively. It includes runnable examples for:
- Differences between launch, async/await, and withContext
- Using try/catch in suspending code
- CoroutineExceptionHandler and when it is invoked
- CancellationException semantics
- A brief supervisor note and link to the dedicated notebook
- Flow exception handling (catch and collector errors)


In [21]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")

## 1) Exceptions in launch vs async vs withContext
- launch: exceptions are uncaught by default in the child; they cancel the parent and are reported to a handler/logging.
- async: exceptions are deferred until await; they are delivered to the awaiting caller.
- withContext: exceptions are rethrown to the caller immediately when the block completes.


In [22]:
import kotlinx.coroutines.*

runBlocking {
    println("[1] launch vs async vs withContext")

    // launch: child throws, parent is canceled; without handler, it crashes the scope
    val job = launch {
        delay(100)
        println("[launch] about to throw")
        throw IllegalStateException("Launch failed")
    }
    try {
        job.join() // join rethrows cancellation (as CancellationException) if parent was canceled
        println("[launch] joined normally (unlikely)")
    } catch (e: CancellationException) {
        println("[launch] parent observed cancellation: ${e::class.simpleName}")
    }
}

[1] launch vs async vs withContext
[launch] about to throw
[launch] parent observed cancellation: JobCancellationException


java.lang.IllegalStateException: Launch failed

In [5]:

runBlocking {
    println("[1] launch vs async vs withContext")

    // async: exception is held and will be thrown when awaiting
    val deferred = async {
        delay(100)
        println("[async] about to throw")
        error("Async failed")
    }
    try {
        deferred.await()
    } catch (e: Throwable) {
        println("[async] caught on await: ${e::class.simpleName} -> ${e.message}")
    }
}


[1] launch vs async vs withContext
[async] about to throw
[async] caught on await: IllegalStateException -> Async failed


java.lang.IllegalStateException: Async failed

In [6]:

runBlocking {
    // withContext: exception is rethrown to the caller of withContext
    try {
        withContext(Dispatchers.Default) {
            delay(50)
            println("[withContext] about to throw")
            throw RuntimeException("withContext failed")
        }
    } catch (e: Throwable) {
        println("[withContext] caller caught: ${e::class.simpleName} -> ${e.message}")
    }
}


[withContext] about to throw
[withContext] caller caught: RuntimeException -> withContext failed


## 2) Using try/catch in coroutines
You can catch exceptions inside a coroutine just like in regular code. For async, place try/catch around await; for launch/withContext, around the block or call site.


In [8]:
import kotlinx.coroutines.*

runBlocking {
    println("[2] try/catch patterns")

    // Catch inside the child
    val job = launch {
        try {
            delay(50)
            throw IllegalArgumentException("Bad arg")
        } catch (e: IllegalArgumentException) {
            println("[try/catch-child] handled: ${e.message}")
        }
    }
    job.join()
}

[2] try/catch patterns
[try/catch-child] handled: Bad arg


In [20]:
runBlocking {
    // Catch around await for async; use a safe exception type
    val d = async {
        delay(50)
        throw IllegalStateException("No access") // replaced java.lang.IllegalAccessException
    }
    try {
        d.await()
    } catch (e: Throwable) {
        println("[try/catch-await] handled: ${e.message}")
    }
}

[try/catch-await] handled: No access


java.lang.IllegalStateException: No access

In [18]:

runBlocking {

    // Catch around withContext call
    try {
        withContext(Dispatchers.Default) {
            throw UnsupportedOperationException("Nope")
        }
    } catch (e: Throwable) {
        println("[try/catch-withContext] handled: ${e::class.simpleName}")
    }
}


[try/catch-withContext] handled: UnsupportedOperationException


## 3) CoroutineExceptionHandler: when it works and when it doesn't
CoroutineExceptionHandler observes uncaught exceptions in a root launch (a coroutine that has no parent that handles the failure). It does not catch:
- Exceptions from async (they are delivered on await)
- Exceptions you already catch with try/catch
- CancellationException (normal cancellation)

Attach it to the CoroutineScope or to the launch call.


In [23]:
import kotlinx.coroutines.*

val handler = CoroutineExceptionHandler { _, e ->
    println("[handler] Caught: ${e::class.simpleName} -> ${e.message}")
}

runBlocking {
    println("[3] CoroutineExceptionHandler demos")

    // Works for launch when exception is uncaught inside child
    val j1 = launch(handler) {
        delay(50)
        println("[handler-demo] launch throwing...")
        throw IllegalStateException("Boom in launch")
    }
    j1.join()

    // Does NOT get called for async
    val d1 = async(handler) {
        delay(50)
        println("[handler-demo] async throwing...")
        throw IllegalStateException("Boom in async")
    }
    try { d1.await() } catch (e: Throwable) {
        println("[handler-demo] async exception observed on await: ${e::class.simpleName}")
    }
}


[3] CoroutineExceptionHandler demos
[handler-demo] launch throwing...


java.lang.IllegalStateException: Boom in launch

## 4) CancellationException is special
- Cancellation is not an error condition; it's a normal control flow signal.
- CancellationException is ignored by CoroutineExceptionHandler and usually not logged.
- When a coroutine is canceled, suspending functions throw CancellationException.


In [24]:
import kotlinx.coroutines.*

runBlocking {
    println("[4] Cancellation semantics")

    val handlerCancel = CoroutineExceptionHandler { _, e ->
        println("[handler-cancel] SHOULD NOT SEE: ${e::class.simpleName}")
    }

    val job = launch(handlerCancel) {
        try {
            repeat(5) { i ->
                delay(100)
                println("[cancel] working $i")
            }
        } finally {
            println("[cancel] finally runs on cancellation")
        }
    }
    delay(220)
    println("[cancel] Cancelling job...")
    job.cancel() // throws CancellationException inside the coroutine
    job.join()
    println("[cancel] Job canceled and joined")
}


[4] Cancellation semantics
[cancel] working 0
[cancel] working 1
[cancel] Cancelling job...
[cancel] finally runs on cancellation
[cancel] Job canceled and joined


## 5) Supervisors and isolation (quick note)
If you need sibling isolation, use supervisorScope or a CoroutineScope(SupervisorJob()). See the dedicated notebook for deeper discussion:
- Coroutines/supervisor_job_and_scope.ipynb

Below is a tiny supervisorScope example to show that one child failing doesn't cancel its sibling; the failure is rethrown after siblings complete.


In [25]:
import kotlinx.coroutines.*

runBlocking {
    try {
        supervisorScope {
            launch { delay(50); println("[supervisor] A throwing"); throw RuntimeException("A failed") }
            launch { delay(120); println("[supervisor] B completed") }
        }
    } catch (e: Throwable) {
        println("[supervisor] parent saw failure: ${e::class.simpleName}")
    }
}


[supervisor] A throwing
[supervisor] B completed


StandaloneCoroutine{Completed}@16fa6808

## 6) Flow exception handling
- Exceptions happening inside the flow builder (upstream) can be handled with the catch operator.
- Exceptions in the collector (downstream) are not handled by catch; use try/catch around collect.


In [28]:
import kotlinx.coroutines.*
import kotlinx.coroutines.flow.*

fun flakyFlow(): Flow<Int> = flow {
    emit(1)
    emit(2)
    throw IllegalStateException("Flow failed upstream")
}

runBlocking {
    println("[6] Flow catch vs collector try/catch")

    // Handle upstream errors with catch
    flakyFlow()
        .catch { e -> println("[flow] caught upstream: ${e::class.simpleName}") }
        .onEach { println("[flow] onEach: $it") }
        .collect()

    // Collector-side exception (e.g., mapping/printing) must be handled outside catch
    try {
        (1..3).asFlow()
            .onEach { if (it == 2) throw ArithmeticException("Downstream issue") }
            .catch { e -> println("[flow] catch will NOT see downstream: ${e::class.simpleName}") }
            .collect { v -> println("[flow] collected $v") }
    } catch (e: Throwable) {
        println("[flow] collector caught downstream: ${e::class.simpleName}")
    }
}


[6] Flow catch vs collector try/catch
[flow] onEach: 1
[flow] onEach: 2
[flow] caught upstream: IllegalStateException
[flow] collected 1
[flow] catch will NOT see downstream: ArithmeticException


## 7) Practical tips
- Prefer handling expected failures where they occur (inside the coroutine) with try/catch.
- For fire-and-forget tasks, attach a CoroutineExceptionHandler to your launch or scope.
- Remember: async exceptions surface on await; never forget to await or properly cancel.
- CancellationException indicates normal cancellation; avoid treating it as an error.
- Use supervisors to isolate siblings when appropriate; still add logging/handlers for visibility.
